In [33]:
import pandas as pd
pd.set_option('display.max_columns', None)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
# Generic inputs for most ML tasks
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn import tree
# import graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

# setup interactive notebook mode
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display, HTML
from sklearn.metrics import accuracy_score, classification_report


In [34]:
df = pd.read_csv("./data/modified/weather_arrivals_departures_merged_correct_tz.csv")
# df['arrival_status'] = pd.cut(df['Arrival Delay (Minutes)'], bins=[float('-inf'), -5, 5.00001, float('inf')], labels=["early", "ontime", "late"], right=False)
df

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Origin Airport,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Taxi-In time (Minutes),Delay Carrier (Minutes)_x,Delay Weather (Minutes)_x,Delay National Aviation System (Minutes)_x,Delay Security (Minutes)_x,Delay Late Aircraft Arrival (Minutes)_x,scheduled_arrival_time,actual_arrival_time,date_x,date_time_x,app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,timestamp_local_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,key_x,date_time_map_x,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Taxi-Out time (Minutes),Delay Carrier (Minutes)_y,Delay Weather (Minutes)_y,Delay National Aviation System (Minutes)_y,Delay Security (Minutes)_y,Delay Late Aircraft Arrival (Minutes)_y,scheduled_departure_time,actual_departure_time,date_y,date_time_y,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,timestamp_local_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,key_y,date_time_map_y,arrival_status
0,B6,01/01/2020,116,JFK,08:22,08:08,75,62,-14,3,0,0,0,0,0,2020-01-01 08:22:00,2020-01-01 08:08:00,2020-01-01,2020-01-01 08:22:00,-6.50,100,-6.20,989,65,-0.40,2020-01-01 08:15:00,0.40,16,804,250,12.10,7.07,SYR,2020-01-01 08:15:00,SYR,07:07,07:06,75,62,-1,11,0,0,0,0,0,2020-01-01 07:07:00,2020-01-01 07:06:00,2020-01-01,2020-01-01 07:07:00,-2.60,87,-2.30,1005,69,2.80,2020-01-01 07:00:00,0.00,16,804,280,11.10,7.70,JFK,2020-01-01 07:00:00,early
1,OO,01/01/2020,3247,ORD,10:43,10:40,113,110,-3,4,0,0,0,0,0,2020-01-01 10:43:00,2020-01-01 10:40:00,2020-01-01,2020-01-01 10:43:00,-5.00,100,-5.20,990,65,0.60,2020-01-01 10:45:00,0.60,16,804,255,14.90,6.70,SYR,2020-01-01 10:45:00,SYR,07:50,07:50,113,110,0,28,0,0,0,0,0,2020-01-01 08:50:00,2020-01-01 08:50:00,2020-01-01,2020-01-01 07:50:00,-12.20,25,-7.90,984,87,-6.10,2020-01-01 07:45:00,1.00,16,802,200,6.60,4.59,ORD,2020-01-01 07:45:00,ontime
2,9E,01/01/2020,5066,JFK,11:37,11:03,92,72,-34,5,0,0,0,0,0,2020-01-01 11:37:00,2020-01-01 11:03:00,2020-01-01,2020-01-01 11:37:00,-5.10,100,-5.40,990,63,0.80,2020-01-01 11:30:00,0.60,16,804,265,12.60,7.75,SYR,2020-01-01 11:30:00,SYR,10:05,09:51,92,72,-14,20,0,0,0,0,0,2020-01-01 10:05:00,2020-01-01 09:51:00,2020-01-01,2020-01-01 10:05:00,-1.20,87,-5.80,1007,49,3.90,2020-01-01 10:00:00,0.70,16,804,270,11.60,7.70,JFK,2020-01-01 10:00:00,early
3,F9,01/01/2020,1214,MCO,14:39,14:23,164,156,-16,6,0,0,0,0,0,2020-01-01 14:39:00,2020-01-01 14:23:00,2020-01-01,2020-01-01 14:39:00,-4.40,100,-5.60,991,60,1.30,2020-01-01 14:45:00,0.50,16,804,260,12.00,7.32,SYR,2020-01-01 14:45:00,SYR,11:55,11:47,164,156,-8,14,0,0,0,0,0,2020-01-01 11:55:00,2020-01-01 11:47:00,2020-01-01,2020-01-01 11:55:00,18.30,25,2.60,1016,35,18.30,2020-01-01 12:00:00,4.00,16,802,290,5.20,3.10,MCO,2020-01-01 12:00:00,early
4,OO,01/01/2020,3253,ORD,14:43,14:38,113,110,-5,5,0,0,0,0,0,2020-01-01 14:43:00,2020-01-01 14:38:00,2020-01-01,2020-01-01 14:43:00,-4.40,100,-5.60,991,60,1.30,2020-01-01 14:45:00,0.50,16,804,260,12.00,7.32,SYR,2020-01-01 14:45:00,SYR,11:50,11:48,113,110,-2,25,0,0,0,0,0,2020-01-01 12:50:00,2020-01-01 12:48:00,2020-01-01,2020-01-01 11:50:00,-2.80,25,-5.70,982,58,1.70,2020-01-01 11:45:00,1.80,16,802,200,10.80,5.09,ORD,2020-01-01 11:45:00,ontime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,YX,12/31/2023,3719,ORD,16:44,17:18,124,157,34,7,0,0,34,0,0,2023-12-31 16:44:00,2023-12-31 17:18:00,2023-12-31,2023-12-31 16:44:00,-5.30,100,-3.40,992,88,-1.70,2023-12-31 16:45:00,0.00,14,804,270,2.72,2.72,SYR,2023-12-31 16:45:00,SYR,13:40,13:41,124,157,1,58,0,0,34,0,0,2023-12-31 14:40:00,2023-12-31 14:41:00,2023-12-31,2023-12-31 13:40:0

In [35]:
df["scheduled_departure_time"] = pd.to_datetime(df["scheduled_departure_time"])
df["actual_departure_time"] = pd.to_datetime(df["actual_departure_time"])


df["scheduled_arrival_time"] = pd.to_datetime(df["scheduled_arrival_time"])
df["actual_arrival_time"] = pd.to_datetime(df["actual_arrival_time"])


In [36]:
# Adding 1hr to ORD to convert to EST from CST
# df.loc[df["Origin Airport"]=="ORD", "scheduled_departure_time"] += pd.Timedelta(hours=1)
# df.loc[df["Origin Airport"]=="ORD", "actual_departure_time"] += pd.Timedelta(hours=1)


In [37]:
# df["diff1"] = (df["scheduled_arrival_time"]-df["scheduled_departure_time"]).dt.total_seconds() // 60
# df["diff2"] = (df["actual_arrival_time"]-df["actual_departure_time"]).dt.total_seconds() // 60


# df[df["Scheduled Elapsed Time (Minutes)"] != df["diff"]][["Scheduled Arrival Time", "scheduled_arrival_time", "Scheduled departure time", "scheduled_departure_time", "Scheduled elapsed time (Minutes)", "Scheduled Elapsed Time (Minutes)", "diff"]]
# df[df["Scheduled Elapsed Time (Minutes)"] != df["diff1"]][["Scheduled Arrival Time", "scheduled_arrival_time", "Scheduled departure time", "scheduled_departure_time", "Scheduled elapsed time (Minutes)", "Scheduled Elapsed Time (Minutes)", "diff1"]]
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff2"]) & (df["Actual Elapsed Time (Minutes)"] != 0)][["Actual Arrival Time", "Actual departure time", 'Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', "Actual elapsed time (Minutes)", "Actual Elapsed Time (Minutes)", "diff2", "actual_departure_time", "actual_arrival_time"]]
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] != 0)][["Actual Arrival Time", "Actual departure time", 'Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', "Actual elapsed time (Minutes)", "Actual Elapsed Time (Minutes)", "diff", "actual_departure_time", "actual_arrival_time"]]
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] != 0) & (df["Actual Elapsed Time (Minutes)"] == df["diff2"])][["Actual Arrival Time", "Actual departure time", 'Carrier Code', 'Date (MM/DD/YYYY)', 'Flight Number', "Actual elapsed time (Minutes)", "Actual Elapsed Time (Minutes)", "diff", "diff2", "actual_departure_time", "actual_arrival_time"]]
# df.loc[df["Scheduled Elapsed Time (Minutes)"] != df["diff1"], "scheduled_arrival_time"] += pd.Timedelta(days=1)
# df.loc[(df["Actual Elapsed Time (Minutes)"] != df["diff2"]) & (df["Actual Elapsed Time (Minutes)"] != 0), "actual_arrival_time"] += pd.Timedelta(days=1)

# df.drop(columns="diff1", inplace=True)
# df.drop(columns="diff2", inplace=True)
# df["diff2"] = df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] != 0)]["diff"] + 1440



In [38]:
 # df[(df["Actual Elapsed Time (Minutes)"]==0)]

In [39]:
# pd.concat([incor, df[df["Actual elapsed time (Minutes)"]==0]]).reset_index(drop=True).to_csv("./data/modified/incorrect_flight_data.csv", index=False)

In [40]:
# df[(df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] == 0)].to_csv("./data/modified/incorrect_flight_data.csv")
# df = df[~((df["Actual Elapsed Time (Minutes)"] != df["diff"]) & (df["Actual Elapsed Time (Minutes)"] == 0))]
# df = df[~(df["Actual Elapsed Time (Minutes)"]==0)].reset_index(drop=True)
# df
# df.drop(columns="diff", inplace=True)
# df.to_csv("./data/modified/weather_arrivals_departures_merged_correct_tz.csv", index=False)

In [41]:
# df

In [42]:
numeric_cols = df.select_dtypes(include='number').columns
numeric_cols

Index(['Flight Number', 'Scheduled Elapsed Time (Minutes)',
       'Actual Elapsed Time (Minutes)', 'Arrival Delay (Minutes)',
       'Taxi-In time (Minutes)', 'Delay Carrier (Minutes)_x',
       'Delay Weather (Minutes)_x',
       'Delay National Aviation System (Minutes)_x',
       'Delay Security (Minutes)_x', 'Delay Late Aircraft Arrival (Minutes)_x',
       'app_temp_x', 'clouds_x', 'dewpt_x', 'pres_x', 'rh_x', 'temp_x', 'uv_x',
       'vis_x', 'weather.code_x', 'wind_dir_x', 'wind_gust_spd_x',
       'wind_spd_x', 'Scheduled elapsed time (Minutes)',
       'Actual elapsed time (Minutes)', 'Departure delay (Minutes)',
       'Taxi-Out time (Minutes)', 'Delay Carrier (Minutes)_y',
       'Delay Weather (Minutes)_y',
       'Delay National Aviation System (Minutes)_y',
       'Delay Security (Minutes)_y', 'Delay Late Aircraft Arrival (Minutes)_y',
       'app_temp_y', 'clouds_y', 'dewpt_y', 'pres_y', 'rh_y', 'temp_y', 'uv_y',
       'vis_y', 'weather.code_y', 'wind_dir_y', 'wind_gus

In [43]:
cat_cols = df.select_dtypes(exclude='number').columns
cat_cols

Index(['Carrier Code', 'Date (MM/DD/YYYY)', 'Origin Airport',
       'Scheduled Arrival Time', 'Actual Arrival Time',
       'scheduled_arrival_time', 'actual_arrival_time', 'date_x',
       'date_time_x', 'timestamp_local_x', 'key_x', 'date_time_map_x',
       'Destination Airport', 'Scheduled departure time',
       'Actual departure time', 'scheduled_departure_time',
       'actual_departure_time', 'date_y', 'date_time_y', 'timestamp_local_y',
       'key_y', 'date_time_map_y', 'arrival_status'],
      dtype='object')

In [44]:
# df.drop(columns=["Date (MM/DD/YYYY)", "Scheduled Arrival Time", "Actual Elapsed Time (Minutes)", "Taxi-In time (Minutes)", "Delay Carrier (Minutes)_x", "Delay Weather (Minutes)_x", "Delay National Aviation System (Minutes)_x", "Delay Security (Minutes)_x", "Delay Late Aircraft Arrival (Minutes)_x", "actual_arrival_time", "date_x", "date_time_x", "date_time_map_x", "key_x", "Destination Airport", "Scheduled departure time", ""])

cat_drop_cols = ["Date (MM/DD/YYYY)", "Scheduled Arrival Time", "Actual Arrival Time", "date_x", 
             "date_time_x", "timestamp_local_x", "key_x", "date_time_map_x", "Destination Airport", 
             "Scheduled departure time", "Actual departure time",'date_y', 'date_time_y', 'timestamp_local_y',
             'key_y', 'date_time_map_y' 
            ]

numeric_drop_cols = ['Flight Number', 'Actual Elapsed Time (Minutes)',
                     'Taxi-In time (Minutes)', 'Delay Carrier (Minutes)_x','Delay Weather (Minutes)_x', 
                     'Delay National Aviation System (Minutes)_x', 'Delay Security (Minutes)_x', 'Delay Late Aircraft Arrival (Minutes)_x',
                     'Scheduled elapsed time (Minutes)','Actual elapsed time (Minutes)', 'Departure delay (Minutes)','Taxi-Out time (Minutes)', 
                     'Delay Carrier (Minutes)_y','Delay Weather (Minutes)_y','Delay National Aviation System (Minutes)_y',
                     'Delay Security (Minutes)_y', 'Delay Late Aircraft Arrival (Minutes)_y', 'Arrival Delay (Minutes)']

In [45]:
model_df = df.drop(columns=cat_drop_cols + numeric_drop_cols + ["actual_arrival_time", "actual_departure_time"]).copy()
model_df

,Carrier Code,Origin Airport,Scheduled Elapsed Time (Minutes),scheduled_arrival_time,app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,scheduled_departure_time,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,arrival_status
0,B6,JFK,75,2020-01-01 08:22:00,-6.50,100,-6.20,989,65,-0.40,0.40,16,804,250,12.10,7.07,2020-01-01 07:07:00,-2.60,87,-2.30,1005,69,2.80,0.00,16,804,280,11.10,7.70,early
1,OO,ORD,113,2020-01-01 10:43:00,-5.00,100,-5.20,990,65,0.60,0.60,16,804,255,14.90,6.70,2020-01-01 08:50:00,-12.20,25,-7.90,984,87,-6.10,1.00,16,802,200,6.60,4.59,ontime
2,9E,JFK,92,2020-01-01 11:37:00,-5.10,100,-5.40,990,63,0.80,0.60,16,804,265,12.60,7.75,2020-01-01 10:05:00,-1.20,87,-5.80,1007,49,3.90,0.70,16,804,270,11.60,7.70,early
3,F9,MCO,164,2020-01-01 14:39:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 11:55:00,18.30,25,2.60,1016,35,18.30,4.00,16,802,290,5.20,3.10,early
4,OO,ORD,113,2020-01-01 14:43:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 12:50:00,-2.80,25,-5.70,982,58,1.70,1.80,16,802,200,10.80,5.09,ontime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,YX,ORD,124,2023-12-31 16:44:00,-5.30,100,-3.40,992,88,-1.70,0.00,14,804,270,2.72,2.72,2023-12-31 14:40:00,-2.40,100,-2.10,989,80,1.00,0.50,4,804,310,7.40,3.10,late
11776,MQ,ORD,106,2023-12-31 17:28:00,-4.50,100,-3.70,991,86,-1.70,0.00,15,804,355,2.04,2.04,2023-12-31 15:42:00,-2.30,100,-2.30,989,78,1.10,0.40,10,804,300,6.60,3.10,late
11777,B6,MCO,175,2023-12-31 19:27:00,-3.40,100,-4.00,1000,86,-1.90,0.00,15,804,360,2.40,1.20,2023-12-31 16:32:00,18.10,80,8.80,1015,52,18.90,1.00,16,804,305,4.40,4.15,early
11778,UA,ORD,114,2023-12-31 21:09:00,-4.30,100,-3.90,1000,88,-2.20,0.00,16,804,360,3.30,1.50,2023-12-31 19:15:00,-5.40,87,-2.30,993,82,0.40,0.00,13,804,315,10.70,7.03,early


In [46]:
initial_df = pd.read_csv("./data/modified/initial_predictions.csv")
initial_df.shape

(23, 29)

In [47]:
# model_df = df.drop(columns=cat_drop_cols+numeric_drop_cols).copy()
# model_df.to_csv("./data/modified/test.csv", index=False)
# carrier_code_map = {
#     "JFK": {
#         "9E": "DL"
#     },
#     "MCO": {
        
#     },
#     "ORD": {
        
#     }
# }

# model_df.loc[(model_df["Origin Airport"]=="JFK")&(), "Carrier Code"] = ""
# model_df.loc[(model_df["Origin Airport"]=="MCO")&(), "Carrier Code"] = ""
# model_df.loc[(model_df["Origin Airport"]=="ORD")&(), "Carrier Code"] = ""


In [48]:
initial_df.columns

Index(['Origin Airport', 'scheduled_departure_time', 'scheduled_arrival_time',
       'Carrier Code', 'app_temp_x', 'clouds_x', 'dewpt_x', 'pres_x', 'rh_x',
       'temp_x', 'uv_x', 'vis_x', 'weather.code_x', 'wind_dir_x',
       'wind_gust_spd_x', 'wind_spd_x', 'app_temp_y', 'clouds_y', 'dewpt_y',
       'pres_y', 'rh_y', 'temp_y', 'uv_y', 'vis_y', 'weather.code_y',
       'wind_dir_y', 'wind_gust_spd_y', 'wind_spd_y',
       'Scheduled Elapsed Time (Minutes)'],
      dtype='object')

In [49]:
numeric_mod_cols = model_df.select_dtypes(include="number").columns
cat_mod_cols = model_df.select_dtypes(exclude="number").columns

In [50]:
model_df

,Carrier Code,Origin Airport,Scheduled Elapsed Time (Minutes),scheduled_arrival_time,app_temp_x,clouds_x,dewpt_x,pres_x,rh_x,temp_x,uv_x,vis_x,weather.code_x,wind_dir_x,wind_gust_spd_x,wind_spd_x,scheduled_departure_time,app_temp_y,clouds_y,dewpt_y,pres_y,rh_y,temp_y,uv_y,vis_y,weather.code_y,wind_dir_y,wind_gust_spd_y,wind_spd_y,arrival_status
0,B6,JFK,75,2020-01-01 08:22:00,-6.50,100,-6.20,989,65,-0.40,0.40,16,804,250,12.10,7.07,2020-01-01 07:07:00,-2.60,87,-2.30,1005,69,2.80,0.00,16,804,280,11.10,7.70,early
1,OO,ORD,113,2020-01-01 10:43:00,-5.00,100,-5.20,990,65,0.60,0.60,16,804,255,14.90,6.70,2020-01-01 08:50:00,-12.20,25,-7.90,984,87,-6.10,1.00,16,802,200,6.60,4.59,ontime
2,9E,JFK,92,2020-01-01 11:37:00,-5.10,100,-5.40,990,63,0.80,0.60,16,804,265,12.60,7.75,2020-01-01 10:05:00,-1.20,87,-5.80,1007,49,3.90,0.70,16,804,270,11.60,7.70,early
3,F9,MCO,164,2020-01-01 14:39:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 11:55:00,18.30,25,2.60,1016,35,18.30,4.00,16,802,290,5.20,3.10,early
4,OO,ORD,113,2020-01-01 14:43:00,-4.40,100,-5.60,991,60,1.30,0.50,16,804,260,12.00,7.32,2020-01-01 12:50:00,-2.80,25,-5.70,982,58,1.70,1.80,16,802,200,10.80,5.09,ontime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,YX,ORD,124,2023-12-31 16:44:00,-5.30,100,-3.40,992,88,-1.70,0.00,14,804,270,2.72,2.72,2023-12-31 14:40:00,-2.40,100,-2.10,989,80,1.00,0.50,4,804,310,7.40,3.10,late
11776,MQ,ORD,106,2023-12-31 17:28:00,-4.50,100,-3.70,991,86,-1.70,0.00,15,804,355,2.04,2.04,2023-12-31 15:42:00,-2.30,100,-2.30,989,78,1.10,0.40,10,804,300,6.60,3.10,late
11777,B6,MCO,175,2023-12-31 19:27:00,-3.40,100,-4.00,1000,86,-1.90,0.00,15,804,360,2.40,1.20,2023-12-31 16:32:00,18.10,80,8.80,1015,52,18.90,1.00,16,804,305,4.40,4.15,early
11778,UA,ORD,114,2023-12-31 21:09:00,-4.30,100,-3.90,1000,88,-2.20,0.00,16,804,360,3.30,1.50,2023-12-31 19:15:00,-5.40,87,-2.30,993,82,0.40,0.00,13,804,315,10.70,7.03,early


In [51]:
# model_df.to_csv("./data/modified/final_data.csv")

In [52]:
model_df.columns

Index(['Carrier Code', 'Origin Airport', 'Scheduled Elapsed Time (Minutes)',
       'scheduled_arrival_time', 'app_temp_x', 'clouds_x', 'dewpt_x', 'pres_x',
       'rh_x', 'temp_x', 'uv_x', 'vis_x', 'weather.code_x', 'wind_dir_x',
       'wind_gust_spd_x', 'wind_spd_x', 'scheduled_departure_time',
       'app_temp_y', 'clouds_y', 'dewpt_y', 'pres_y', 'rh_y', 'temp_y', 'uv_y',
       'vis_y', 'weather.code_y', 'wind_dir_y', 'wind_gust_spd_y',
       'wind_spd_y', 'arrival_status'],
      dtype='object')

In [53]:
set(model_df.columns.tolist())-set(initial_df.columns.tolist())

{'arrival_status'}

In [54]:
set(initial_df.columns.tolist())-set(model_df.columns.tolist())

set()

In [55]:
model_df.dtypes

Carrier Code                                object
Origin Airport                              object
Scheduled Elapsed Time (Minutes)             int64
scheduled_arrival_time              datetime64[ns]
app_temp_x                                 float64
clouds_x                                     int64
dewpt_x                                    float64
pres_x                                       int64
rh_x                                         int64
temp_x                                     float64
uv_x                                       float64
vis_x                                        int64
weather.code_x                               int64
wind_dir_x                                   int64
wind_gust_spd_x                            float64
wind_spd_x                                 float64
scheduled_departure_time            datetime64[ns]
app_temp_y                                 float64
clouds_y                                     int64
dewpt_y                        

In [22]:
def add_custom_features(model_df):
    model_df['scheduled_departure_hour'] = model_df['scheduled_departure_time'].dt.hour
    model_df['scheduled_departure_day'] = model_df['scheduled_departure_time'].dt.day
    model_df['scheduled_departure_month'] = model_df['scheduled_departure_time'].dt.month
    model_df['scheduled_departure_weekday'] = model_df['scheduled_departure_time'].dt.weekday

    model_df['scheduled_arrival_hour'] = model_df['scheduled_arrival_time'].dt.hour
    model_df['scheduled_arrival_day'] = model_df['scheduled_arrival_time'].dt.day
    model_df['scheduled_arrival_month'] = model_df['scheduled_arrival_time'].dt.month
    model_df['scheduled_arrival_weekday'] = model_df['scheduled_arrival_time'].dt.weekday

    weather_features = ['app_temp', 'clouds', 'dewpt', 'pres', 'rh', 'temp', 'uv', 'vis','weather.code', 'wind_dir', 'wind_gust_spd', 'wind_spd']
    # weather_features = ['app_temp', 'dewpt', 'precip_rate', 'rh', 'temp', 'uv', 'vis', 'weather.code', 'wind_gust_spd', 'wind_spd', 'snow_rate']
    for feature in weather_features:
        model_df[f'{feature}_diff'] = model_df[f'{feature}_x'] - model_df[f'{feature}_y']


    carrier_dummies = pd.get_dummies(model_df['Carrier Code'], prefix='carrier')
    airport_dummies = pd.get_dummies(model_df['Origin Airport'], prefix='airport')


    model_df = pd.concat([model_df, carrier_dummies, airport_dummies], axis=1)

    model_df['dep_time_day_prop'] = model_df['scheduled_departure_time'].dt.hour * 60 + model_df['scheduled_departure_time'].dt.minute
    model_df['dep_time_day_prop'] /= 1440  

    model_df['arr_time_day_prop'] = model_df['scheduled_arrival_time'].dt.hour * 60 + model_df['scheduled_arrival_time'].dt.minute
    model_df['arr_time_day_prop'] /= 1440  


    model_df['night_flight'] = model_df['scheduled_departure_time'].dt.hour.apply(lambda x: 1 if x >= 20 or x < 6 else 0)

    model_df['extreme_weather_temp_low'] = model_df['temp_x'].apply(lambda x: 1 if x < -10 else 0)
    model_df['extreme_weather_temp_high'] = model_df['temp_x'].apply(lambda x: 1 if x > 35 else 0)
    model_df['extreme_weather_wind'] = model_df['wind_gust_spd_x'].apply(lambda x: 1 if x > 50 else 0)
    
    model_df['time_of_day'] = model_df['scheduled_arrival_hour'].apply(categorize_hour)

    model_df["peak_zone"] = "peak"

    model_df.loc[((model_df['time_of_day']=="Morning") | (model_df['time_of_day']=="Early Morning") | (model_df['time_of_day']=="Afternoon")), "peak_zone"] = "off_peak"
    
    categorical_cols = ['time_of_day', 'peak_zone']
    model_df = pd.get_dummies(model_df, columns=categorical_cols)
    bool_cols = model_df.select_dtypes(include=["bool"]).columns
    for i in bool_cols:
        model_df[i] *= 1
    drop_cols = model_df.select_dtypes(exclude=["number"]).columns.to_list()

    
    return model_df

def categorize_hour(hour):
    if 0 <= hour <= 3:
        return 'Early Morning'
    elif 4 <= hour <= 7:
        return 'Morning'
    elif 8 <= hour <= 11:
        return 'Late Morning'
    elif 12 <= hour <= 15:
        return 'Afternoon'
    elif 16 <= hour <= 19:
        return 'Evening'
    elif 20 <= hour <= 23:
        return 'Night'





,carrier_9E,carrier_AA,carrier_B6,carrier_F9,carrier_MQ,carrier_OO,carrier_UA,carrier_WN,carrier_YX,airport_JFK,airport_MCO,airport_ORD,time_of_day_Afternoon,time_of_day_Early Morning,time_of_day_Evening,time_of_day_Late Morning,time_of_day_Morning,time_of_day_Night,peak_zone_off_peak,peak_zone_peak
0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1
2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1
11776,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
11777,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
11778,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1


In [22]:

# model_df['scheduled_departure_hour'] = model_df['scheduled_departure_time'].dt.hour
# model_df['scheduled_departure_day'] = model_df['scheduled_departure_time'].dt.day
# model_df['scheduled_departure_month'] = model_df['scheduled_departure_time'].dt.month
# model_df['scheduled_departure_weekday'] = model_df['scheduled_departure_time'].dt.weekday

# model_df['scheduled_arrival_hour'] = model_df['scheduled_arrival_time'].dt.hour
# model_df['scheduled_arrival_day'] = model_df['scheduled_arrival_time'].dt.day
# model_df['scheduled_arrival_month'] = model_df['scheduled_arrival_time'].dt.month
# model_df['scheduled_arrival_weekday'] = model_df['scheduled_arrival_time'].dt.weekday

# weather_features = ['app_temp', 'clouds', 'dewpt', 'pres', 'rh', 'temp', 'uv', 'vis','weather.code', 'wind_dir', 'wind_gust_spd', 'wind_spd']
# # weather_features = ['app_temp', 'dewpt', 'precip_rate', 'rh', 'temp', 'uv', 'vis', 'weather.code', 'wind_gust_spd', 'wind_spd', 'snow_rate']
# for feature in weather_features:
#     model_df[f'{feature}_diff'] = model_df[f'{feature}_x'] - model_df[f'{feature}_y']


# carrier_dummies = pd.get_dummies(model_df['Carrier Code'], prefix='carrier')
# airport_dummies = pd.get_dummies(model_df['Origin Airport'], prefix='airport')


# model_df = pd.concat([model_df, carrier_dummies, airport_dummies], axis=1)

# model_df['dep_time_day_prop'] = model_df['scheduled_departure_time'].dt.hour * 60 + model_df['scheduled_departure_time'].dt.minute
# model_df['dep_time_day_prop'] /= 1440  

# model_df['arr_time_day_prop'] = model_df['scheduled_arrival_time'].dt.hour * 60 + model_df['scheduled_arrival_time'].dt.minute
# model_df['arr_time_day_prop'] /= 1440  


# model_df['night_flight'] = model_df['scheduled_departure_time'].dt.hour.apply(lambda x: 1 if x >= 20 or x < 6 else 0)

# model_df['extreme_weather_temp_low'] = model_df['temp_x'].apply(lambda x: 1 if x < -10 else 0)
# model_df['extreme_weather_temp_high'] = model_df['temp_x'].apply(lambda x: 1 if x > 35 else 0)
# model_df['extreme_weather_wind'] = model_df['wind_gust_spd_x'].apply(lambda x: 1 if x > 50 else 0)

# def categorize_hour(hour):
#     if 0 <= hour <= 3:
#         return 'Early Morning'
#     elif 4 <= hour <= 7:
#         return 'Morning'
#     elif 8 <= hour <= 11:
#         return 'Late Morning'
#     elif 12 <= hour <= 15:
#         return 'Afternoon'
#     elif 16 <= hour <= 19:
#         return 'Evening'
#     elif 20 <= hour <= 23:
#         return 'Night'

# model_df['time_of_day'] = model_df['scheduled_arrival_hour'].apply(categorize_hour)

# model_df["peak_zone"] = "peak"

# model_df.loc[((model_df['time_of_day']=="Morning") | (model_df['time_of_day']=="Early Morning") | (model_df['time_of_day']=="Afternoon")), "peak_zone"] = "off_peak"
# # model_df['extreme_weather_snow'] = model_df['snow_x'].apply(lambda x: 1 if x > 0.5 else 0)  # More than 0.5 mm/h

# # model_df[['dep_time_day_prop', 'arr_time_day_prop', 'night_flight', 
# #       'extreme_weather_temp_low', 'extreme_weather_temp_high', 'extreme_weather_wind', 'extreme_weather_snow']].head()

# # model_df[['dep_time_day_prop', 'arr_time_day_prop', 'night_flight', 
# #       'extreme_weather_temp_low', 'extreme_weather_temp_high', 'extreme_weather_wind']].head()

# categorical_cols = ['time_of_day', 'peak_zone']
# model_df = pd.get_dummies(model_df, columns=categorical_cols)

# bool_cols = model_df.select_dtypes(include=["bool"]).columns
# for i in bool_cols:
#     model_df[i] *= 1
# model_df[bool_cols]

# bool_cols = model_df.select_dtypes(include=["bool"]).columns

# drop_cols = model_df.select_dtypes(exclude=["number"]).columns.to_list()
# drop_cols.append('Arrival Delay (Minutes)')


,carrier_9E,carrier_AA,carrier_B6,carrier_F9,carrier_MQ,carrier_OO,carrier_UA,carrier_WN,carrier_YX,airport_JFK,airport_MCO,airport_ORD,time_of_day_Afternoon,time_of_day_Early Morning,time_of_day_Evening,time_of_day_Late Morning,time_of_day_Morning,time_of_day_Night,peak_zone_off_peak,peak_zone_peak
0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,1
2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1
3,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
4,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1
11776,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1
11777,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1
11778,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1


In [23]:
# drop_cols = model_df.select_dtypes(exclude=["number"]).columns.to_list()
# drop_cols.append('Arrival Delay (Minutes)')

In [24]:
# selected_features = model_df.select_dtypes(include=["number"]).columns
# len(selected_features)

In [25]:
# Select only numeric columns
numeric_cols = model_df.select_dtypes(include=['number'])

# Calculate correlation with "Arrival Delay (Minutes)_a" and sort by absolute value
corr_with_delay = model_df.corr(numeric_only=True)['Arrival Delay (Minutes)'].abs().sort_values(ascending=False)

# print(corr_with_delay)

# Select top 12 features with low intercorrelation
selected_features = []
for feature in corr_with_delay.index:
    if len(selected_features) >= 20:
        break
    if corr_with_delay[feature] >= -0.1:
        selected_features.append(feature)
        
selected_features.remove('Arrival Delay (Minutes)')
print(selected_features)

# Print remaining numerical columns
remaining_numeric_cols = [col for col in numeric_cols.columns if col not in selected_features]
remaining_numeric_cols.remove('Arrival Delay (Minutes)')
print("Remaining Numerical Columns:")
print(remaining_numeric_cols)

# Remove the specified columns

columns_to_remove = remaining_numeric_cols
# .append('Year')

model_df = model_df.drop(columns=remaining_numeric_cols)

['carrier_B6', 'weather.code_y', 'airport_MCO', 'vis_y', 'carrier_9E', 'Scheduled Elapsed Time (Minutes)', 'rh_y', 'dewpt_y', 'vis_x', 'pres_x', 'dewpt_diff', 'weather.code_diff', 'dep_time_day_prop', 'scheduled_departure_hour', 'app_temp_diff', 'rh_x', 'time_of_day_Late Morning', 'airport_JFK', 'temp_diff']
Remaining Numerical Columns:
['app_temp_x', 'clouds_x', 'dewpt_x', 'temp_x', 'uv_x', 'weather.code_x', 'wind_dir_x', 'wind_gust_spd_x', 'wind_spd_x', 'app_temp_y', 'clouds_y', 'pres_y', 'temp_y', 'uv_y', 'wind_dir_y', 'wind_gust_spd_y', 'wind_spd_y', 'scheduled_departure_day', 'scheduled_departure_month', 'scheduled_departure_weekday', 'scheduled_arrival_hour', 'scheduled_arrival_day', 'scheduled_arrival_month', 'scheduled_arrival_weekday', 'clouds_diff', 'pres_diff', 'rh_diff', 'uv_diff', 'vis_diff', 'wind_dir_diff', 'wind_gust_spd_diff', 'wind_spd_diff', 'carrier_AA', 'carrier_F9', 'carrier_MQ', 'carrier_OO', 'carrier_UA', 'carrier_WN', 'carrier_YX', 'airport_ORD', 'arr_time_day_

In [26]:
remaining_numeric_cols

['app_temp_x',
 'clouds_x',
 'dewpt_x',
 'temp_x',
 'uv_x',
 'weather.code_x',
 'wind_dir_x',
 'wind_gust_spd_x',
 'wind_spd_x',
 'app_temp_y',
 'clouds_y',
 'pres_y',
 'temp_y',
 'uv_y',
 'wind_dir_y',
 'wind_gust_spd_y',
 'wind_spd_y',
 'scheduled_departure_day',
 'scheduled_departure_month',
 'scheduled_departure_weekday',
 'scheduled_arrival_hour',
 'scheduled_arrival_day',
 'scheduled_arrival_month',
 'scheduled_arrival_weekday',
 'clouds_diff',
 'pres_diff',
 'rh_diff',
 'uv_diff',
 'vis_diff',
 'wind_dir_diff',
 'wind_gust_spd_diff',
 'wind_spd_diff',
 'carrier_AA',
 'carrier_F9',
 'carrier_MQ',
 'carrier_OO',
 'carrier_UA',
 'carrier_WN',
 'carrier_YX',
 'airport_ORD',
 'arr_time_day_prop',
 'night_flight',
 'extreme_weather_temp_low',
 'extreme_weather_temp_high',
 'extreme_weather_wind',
 'time_of_day_Afternoon',
 'time_of_day_Early Morning',
 'time_of_day_Evening',
 'time_of_day_Morning',
 'time_of_day_Night',
 'peak_zone_off_peak',
 'peak_zone_peak']

In [27]:
# # Define the target variable based on arrival delay
# df['Class of Arrival Time'] = pd.cut(df['Arrival Delay (Minutes)_a'],
#                                        bins=[float('-inf'), -5, 5, float('inf')],
#                                        labels=['Early', 'On-time', 'Late'])

# Drop unnecessary columns and the target variable
X = model_df.drop(columns=drop_cols, axis=1)
y = model_df['arrival_status']
# y = df['Arrival Delay (Minutes)_a']


# Convert categorical variables to numerical using one-hot encoding
# categorical_cols = ['Origin Airport_a', 'wx_phrase_a', 'uv_desc_a', 'clds_a',
#                     'wx_phrase_d', 'uv_desc_d', 'clds_d', 'Main_Carrier', 'Week_Category', 'A_Time_of_Day']
# X_encoded = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# scaler = StandardScaler()
# X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
# X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

In [28]:
X_encoded = X

# Standardize numerical columns
numerical_cols = selected_features
scaler = StandardScaler()
X_encoded[numerical_cols] = scaler.fit_transform(X_encoded[numerical_cols])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


In [29]:
X_encoded[selected_features]

,carrier_B6,weather.code_y,airport_MCO,vis_y,carrier_9E,Scheduled Elapsed Time (Minutes),rh_y,dewpt_y,vis_x,pres_x,dewpt_diff,weather.code_diff,dep_time_day_prop,scheduled_departure_hour,app_temp_diff,rh_x,time_of_day_Late Morning,airport_JFK,temp_diff
0,1.74,0.28,-0.47,0.31,-0.53,-1.14,0.35,-1.00,0.37,-1.54,-0.05,0.06,-1.86,-1.80,0.07,0.10,2.22,1.26,0.10
1,-0.57,0.25,-0.47,0.31,-0.53,0.13,1.31,-1.54,0.37,-1.41,0.85,0.08,-1.49,-1.58,1.35,0.10,2.22,-0.80,1.44
2,-0.57,0.28,-0.47,0.31,1.89,-0.57,-0.71,-1.34,0.37,-1.41,0.53,0.06,-1.22,-1.14,0.07,-0.01,2.22,1.26,0.11
3,-0.57,0.25,2.15,0.31,-0.53,1.84,-1.45,-0.52,0.37,-1.28,-0.64,0.08,-0.82,-0.92,-2.09,-0.17,-0.45,-0.80,-1.77
4,-0.57,0.25,-0.47,0.31,-0.53,0.13,-0.23,-1.33,0.37,-1.28,0.49,0.08,-0.62,-0.70,0.34,-0.17,-0.45,-0.80,0.47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11775,-0.57,0.28,-0.47,-3.98,-0.53,0.50,0.94,-0.98,-0.28,-1.15,0.30,0.06,-0.23,-0.26,0.19,1.37,-0.45,-0.80,0.16
11776,-0.57,0.28,-0.47,-1.83,-0.53,-0.10,0.83,-1.00,0.05,-1.28,0.29,0.06,-0.00,-0.05,0.27,1.26,-0.45,-0.80,0.15
11777,1.74,0.28,2.15,0.31,-0.53,2.21,-0.55,0.09,0.05,-0.10,-1.27,0.06,0.18,0.17,-1.95,1.26,-0.45,-0.80,-2.29
11778,-0.57,0.28,-0.47,-0.76,-0.53,0.17,1.04,-1.00,0.37,-0.10,0.26,0.06,0.77,0.83,0.65,1.37,-0.45,-0.80,0.18


In [30]:
X_encoded[numerical_cols] = scaler.fit_transform(X_encoded[numerical_cols])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


In [31]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import mean_absolute_error
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import pandas as pd

# rf_model = RandomForestClassifier(random_state=42)
# param_grid = {
#     'n_estimators': [100, 200],  # fewer options
#     'max_depth': [10, 20, None],  # added 'None' to test no maximum depth
#     'min_samples_split': [2, 10],  # fewer and higher values
#     'min_samples_leaf': [1, 4],    # fewer and higher values
#     'bootstrap': [True],           # only bootstrap
#     'criterion': ['gini']          # only one criterion to reduce complexity
# }
# {'n_estimators': 200, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': 20, 'criterion': 'gini', 'bootstrap': True}

# Initialize and train different models
models = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'SVM': SVC(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42,n_estimators = 250,max_depth = 50, learning_rate= 1 ),
    'ada' : AdaBoostClassifier(random_state=42),
    # 'xg':  XGBClassifier(random_state=42),
    'knn' : KNeighborsClassifier(n_neighbors=15),
    'dtree':  DecisionTreeClassifier(max_depth=7,random_state=42),
    'rand_class':  RandomForestClassifier(max_depth=15,random_state=42),
    'GNB':  GaussianNB(),
    'rf_random': RandomForestClassifier(random_state=42, n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_depth=20, criterion='gini', bootstrap=True)
}

for model_name, model in models.items():
    try:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)
        print(f'{model_name} Accuracy: {accuracy:.2f}')
        # mae = mean_absolute_error(y_test, y_pred)
        # mae_ratio = mae / np.mean(y_test)
        # print("MAE:", mae)
        # print("MAE Ratio:", mae_ratio)
        # print(model.score(X_test,y_test))
        # print(classification_report(y_test, y_pred))
        # print('-' * 50)
    except Exception as e:
        print(e)

LogisticRegression(random_state=42)

Logistic Regression Accuracy: 0.55


DecisionTreeClassifier(random_state=42)

Decision Tree Accuracy: 0.45


RandomForestClassifier(random_state=42)

...

rand_class Accuracy: 0.56


GaussianNB()

GNB Accuracy: 0.52


RandomForestClassifier(max_depth=20, min_samples_split=10, n_estimators=200,
                       random_state=42)

rf_random Accuracy: 0.57
